# Coral Analysis

In [1]:
import turicreate

In [2]:
import re

In [4]:
# summary = turicreate.SFrame('crossrefSummary.csv')
# summary = turicreate.SFrame('scholarSummary.csv')
summary = turicreate.SFrame('selectedSummary.csv')

Finished parsing file /home/liz/coral/pdf-analysis/selectedSummary.csv

Parsing completed. Parsed 76 lines in 0.016628 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/liz/coral/pdf-analysis/selectedSummary.csv

Parsing completed. Parsed 76 lines in 0.017815 secs.

In [5]:
summary = summary[summary['Abstract'] != None]  # only search documents where abstract was found

### Search text for all Acropora species (species list taken from Acropora Wiki)

In [6]:
species_list = []
with open('acroporaSpecies.txt', 'r') as f:
    text = f.read()
    species_list = re.findall('(?<=\n)[A-Za-z]+ [a-z]+', text) # pull out species names only
len(species_list)

289

In [7]:
def searchSpecies(text):
    species_found = []
    for species in species_list:
        # e.g. search for Acropora digitifera or A. digitifera 
        #if species in text or re.search(species[0]+'[\.]? '+species.split(' ')[1], text):
        if species in text or re.search(species[0]+'[\.]? '+species.split(' ')[1], text):
            species_found.append(species)
    return species_found

In [8]:
summary['Species present'] = summary['Abstract'].apply(searchSpecies)
summary

Columns:
	ID	str
	DOI	str
	Title	str
	Date	str
	Keywords	str
	Abstract	str
	Language	str
	Species present	list

Rows: 73

Data:
+-------------------------------+-------------------------------+
|               ID              |              DOI              |
+-------------------------------+-------------------------------+
|   10.1038_s41598-018-21267-3  |   10.1038/s41598-018-21267-3  |
| 10.1016_j.marpolbul.2016.0... | 10.1016/j.marpolbul.2016.0... |
|   10.1186_s12864-015-2355-x   |   10.1186/s12864-015-2355-x   |
|       10.1111_mec.15642       |       10.1111/mec.15642       |
|  10.1371_journal.pone.0016095 |  10.1371/journal.pone.0016095 |
|   10.1007_s10126-020-09984-1  |   10.1007/s10126-020-09984-1  |
|  10.1371_journal.pone.0020140 |  10.1371/journal.pone.0020140 |
|  10.1371_journal.pone.0081821 |  10.1371/journal.pone.0081821 |
|         10.1086_692717        |              None             |
|       10.5539_ijb.v4n1p3      |       10.5539/ijb.v4n1p3      |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------+
|             Title             |        Date       |
+-------------------------------+-------------------+
| Elevated CO 2 delays the e... |        None       |
| Proteomic analysis of blea... |   19 April 2016   |
| Functional conservation of... |        None       |
| Local acclimatisation-driv... |        None       |
| Regulation of Apoptotic Me... |  January 24, 2011 |
| Transcriptome Analyses of ... |    21 July 2020   |
| Identification of Fast-Evo... |   June 20, 2011   |
| RNA-seq Profiles of Immune... | November 21, 2013 |
| Early Transcriptional Resp... |    31 May 2017    |
| Effects of Heat Stress on ... |  January 1, 2012  |
+-------------------------------+-------------------+
+-------------------------------+-------------------------------+----------+
|            Keywords           |            Abstract           | Language |
+-------------------------------+-------------------------------+----------+
|              None             | The effects of elevated CO... |    en    |
| Coral bleaching Proteomics... | There has been an increase... |    en    |
| Apoptosis,Caspase,Bcl-2,Co... | Background: Apoptotic cell... |    en    |
| Acropora muricata,bleachin... | Coral reefs are the most p... |    en    |
|              None             | Background: Mass coral ble... |    en    |
| Coral,Vibrio coralliilytic... | Elevated sea surface tempe... |    en    |
|              None             | To identify fast-evolving ... |    en    |
|              None             | Coral diseases are among t... |    en    |
| HSP, heat shock protein,Lo... | Corals respond to heat pul... |    en    |
| Carotenoids,Chlorophyll a ... | The question of whether gl... |    en    |
+-------------------------------+-------------------------------+----------+
+-------------------------------+
|        Species present        |
+-------------------------------+
| [Acropora gemmifera, Acrop... |
|       [Acropora palmata]      |
|      [Acropora millepora]     |
|               []              |
|      [Acropora millepora]     |
|     [Acropora digitifera]     |
| [Acropora millepora, Acrop... |
|     [Acropora cervicornis]    |
|               []              |
|     [Acropora hyacinthus]     |
+-------------------------------+
[73 rows x 8 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [9]:
species_info = turicreate.SFrame({'Species': species_list})

# Count the number of papers referencing a given species
def totalPapersWithSpecies(species):
    return sum([species in species_pres for species_pres in summary['Species present']])
  
species_info['Total papers'] = species_info['Species'].apply(totalPapersWithSpecies)

### Most Commonly Studied Species

In [10]:
species_info = species_info.sort('Total papers', ascending=False)

In [11]:
species_info

Columns:
	Species	str
	Total papers	int

Rows: 289

Data:
+------------------------+--------------+
|        Species         | Total papers |
+------------------------+--------------+
|   Acropora millepora   |      20      |
|    Acropora palmata    |      11      |
|  Acropora digitifera   |      8       |
|    Acropora tenuis     |      7       |
|  Acropora cervicornis  |      3       |
|    Acropora aspera     |      3       |
|  Acropora hyacinthus   |      2       |
|    Acropora nasuta     |      2       |
|    Acropora aculeus    |      1       |
| Acropora microphthalma |      1       |
+------------------------+--------------+
[289 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [12]:
sum(species_info['Total papers'] > 0)  # count how many species appear at all in any paper
species_info[species_info['Total papers'] > 0]['Species']

dtype: str
Rows: ?
['Acropora millepora', 'Acropora palmata', 'Acropora digitifera', 'Acropora tenuis', 'Acropora cervicornis', 'Acropora aspera', 'Acropora hyacinthus', 'Acropora nasuta', 'Acropora aculeus', 'Acropora microphthalma', 'Montipora digitata', 'Acropora gemmifera', 'Acropora muricata', 'Acropora humilis', ... ]

### Search text for all proteins on Uniprot for Acropora

In [13]:
def simplifyProteinName(protein_name):
    protein_name = re.sub(' \(.*\)', '', protein_name)  # remove anything in parentheses, unlikely to match
    protein_name = re.sub('  ', ' ', protein_name)  # get rid of double spaces
    return protein_name

def removeCommonGeneNames(gene_name):
    if len(gene_name) < 4 or re.match('[0-9\.]+', gene_name):
        gene_name = None
    return gene_name

In [14]:
swissprot = turicreate.SFrame.read_csv('swissprot_all.tab', '\t')
swissprot = swissprot.rename({'Gene names  (primary )':'Gene names'})
swissprot['Protein names'] = swissprot['Protein names'].apply(simplifyProteinName)  # make names more searchable
swissprot['Gene names'] = swissprot['Gene names'].apply(removeCommonGeneNames)  # some gene names are common words
swissprot = swissprot['Protein names', 'Gene names'].unique()  # remove duplicates
swissprot = swissprot.swap_columns('Protein names', 'Gene names')
summary = summary[summary['Language'] == 'en']  # only search documents written in english
summary

Successfully parsed 3 tokens: 
	0: Q58251
	1: reviewed
	2: [Methyl-co ... C 2.1.1.-)

Successfully parsed 3 tokens: 
	0: A0A5Q0MUT1
	1: reviewed
	2: [Thr6, Val ... hyllokinin

Successfully parsed 3 tokens: 
	0: L0PJV8
	1: reviewed
	2: [Thr6]-bra ... radykinin]

Successfully parsed 3 tokens: 
	0: P86629
	1: reviewed
	2: [Thr6]-bradykinin

Successfully parsed 3 tokens: 
	0: P86815
	1: reviewed
	2: [Ala1,Thr6 ... ys,Ala,Val

...

Read 531163 lines. Lines per second: 1.38069e+06

43 lines failed to parse correctly

Finished parsing file /home/liz/coral/pdf-analysis/swissprot_all.tab

Parsing completed. Parsed 563929 lines in 0.396998 secs.

Columns:
	ID	str
	DOI	str
	Title	str
	Date	str
	Keywords	str
	Abstract	str
	Language	str
	Species present	list

Rows: Unknown

Data:
+-------------------------------+-------------------------------+
|               ID              |              DOI              |
+-------------------------------+-------------------------------+
|   10.1038_s41598-018-21267-3  |   10.1038/s41598-018-21267-3  |
| 10.1016_j.marpolbul.2016.0... | 10.1016/j.marpolbul.2016.0... |
|   10.1186_s12864-015-2355-x   |   10.1186/s12864-015-2355-x   |
|       10.1111_mec.15642       |       10.1111/mec.15642       |
|  10.1371_journal.pone.0016095 |  10.1371/journal.pone.0016095 |
|   10.1007_s10126-020-09984-1  |   10.1007/s10126-020-09984-1  |
|  10.1371_journal.pone.0020140 |  10.1371/journal.pone.0020140 |
|  10.1371_journal.pone.0081821 |  10.1371/journal.pone.0081821 |
|         10.1086_692717        |              None             |
|       10.5539_ijb.v4n1p3      |       10.5539/ijb.v4n1p3      |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------+
|             Title             |        Date       |
+-------------------------------+-------------------+
| Elevated CO 2 delays the e... |        None       |
| Proteomic analysis of blea... |   19 April 2016   |
| Functional conservation of... |        None       |
| Local acclimatisation-driv... |        None       |
| Regulation of Apoptotic Me... |  January 24, 2011 |
| Transcriptome Analyses of ... |    21 July 2020   |
| Identification of Fast-Evo... |   June 20, 2011   |
| RNA-seq Profiles of Immune... | November 21, 2013 |
| Early Transcriptional Resp... |    31 May 2017    |
| Effects of Heat Stress on ... |  January 1, 2012  |
+-------------------------------+-------------------+
+-------------------------------+-------------------------------+----------+
|            Keywords           |            Abstract           | Language |
+-------------------------------+-------------------------------+----------+
|              None             | The effects of elevated CO... |    en    |
| Coral bleaching Proteomics... | There has been an increase... |    en    |
| Apoptosis,Caspase,Bcl-2,Co... | Background: Apoptotic cell... |    en    |
| Acropora muricata,bleachin... | Coral reefs are the most p... |    en    |
|              None             | Background: Mass coral ble... |    en    |
| Coral,Vibrio coralliilytic... | Elevated sea surface tempe... |    en    |
|              None             | To identify fast-evolving ... |    en    |
|              None             | Coral diseases are among t... |    en    |
| HSP, heat shock protein,Lo... | Corals respond to heat pul... |    en    |
| Carotenoids,Chlorophyll a ... | The question of whether gl... |    en    |
+-------------------------------+-------------------------------+----------+
+-------------------------------+
|        Species present        |
+-------------------------------+
| [Acropora gemmifera, Acrop... |
|       [Acropora palmata]      |
|      [Acropora millepora]     |
|               []              |
|      [Acropora millepora]     |
|     [Acropora digitifera]     |
| [Acropora millepora, Acrop... |
|     [Acropora cervicornis]    |
|               []              |
|     [Acropora hyacinthus]     |
+-------------------------------+
[? rows x 8 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [15]:
def searchGenes(text):
    print(".", end="")
    gene_list = []
    for gene in swissprot:
        gene_name = gene['Gene names']
        protein_name = gene['Protein names']
        if protein_name in text:
            gene_list.append(protein_name)  # found match
        elif gene_name:
            gene_name = re.sub(r'\\', r'\\\\', gene_name)  # escape any backslash characters in gene names
            reg = re.compile('(?<=[^a-zA-Z])'+gene_name+'(?=[^a-zA-Z])') # don't let the gene name be part of a word
            if re.search(reg, text):
                gene_list.append(gene_name)
    return gene_list


In [16]:
# This can take some time (searching for all proteins/genes is swissprot)...
summary['Genes present'] = summary['Abstract'].apply(searchGenes)
summary.materialize()

.........................................................................

In [17]:
summary

Columns:
	ID	str
	DOI	str
	Title	str
	Date	str
	Keywords	str
	Abstract	str
	Language	str
	Species present	list
	Genes present	list

Rows: 73

Data:
+-------------------------------+-------------------------------+
|               ID              |              DOI              |
+-------------------------------+-------------------------------+
|   10.1038_s41598-018-21267-3  |   10.1038/s41598-018-21267-3  |
| 10.1016_j.marpolbul.2016.0... | 10.1016/j.marpolbul.2016.0... |
|   10.1186_s12864-015-2355-x   |   10.1186/s12864-015-2355-x   |
|       10.1111_mec.15642       |       10.1111/mec.15642       |
|  10.1371_journal.pone.0016095 |  10.1371/journal.pone.0016095 |
|   10.1007_s10126-020-09984-1  |   10.1007/s10126-020-09984-1  |
|  10.1371_journal.pone.0020140 |  10.1371/journal.pone.0020140 |
|  10.1371_journal.pone.0081821 |  10.1371/journal.pone.0081821 |
|         10.1086_692717        |              None             |
|       10.5539_ijb.v4n1p3      |       10.5539/ijb.v4n1p3      |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------+
|             Title             |        Date       |
+-------------------------------+-------------------+
| Elevated CO 2 delays the e... |        None       |
| Proteomic analysis of blea... |   19 April 2016   |
| Functional conservation of... |        None       |
| Local acclimatisation-driv... |        None       |
| Regulation of Apoptotic Me... |  January 24, 2011 |
| Transcriptome Analyses of ... |    21 July 2020   |
| Identification of Fast-Evo... |   June 20, 2011   |
| RNA-seq Profiles of Immune... | November 21, 2013 |
| Early Transcriptional Resp... |    31 May 2017    |
| Effects of Heat Stress on ... |  January 1, 2012  |
+-------------------------------+-------------------+
+-------------------------------+-------------------------------+----------+
|            Keywords           |            Abstract           | Language |
+-------------------------------+-------------------------------+----------+
|              None             | The effects of elevated CO... |    en    |
| Coral bleaching Proteomics... | There has been an increase... |    en    |
| Apoptosis,Caspase,Bcl-2,Co... | Background: Apoptotic cell... |    en    |
| Acropora muricata,bleachin... | Coral reefs are the most p... |    en    |
|              None             | Background: Mass coral ble... |    en    |
| Coral,Vibrio coralliilytic... | Elevated sea surface tempe... |    en    |
|              None             | To identify fast-evolving ... |    en    |
|              None             | Coral diseases are among t... |    en    |
| HSP, heat shock protein,Lo... | Corals respond to heat pul... |    en    |
| Carotenoids,Chlorophyll a ... | The question of whether gl... |    en    |
+-------------------------------+-------------------------------+----------+
+-------------------------------+-------------------------------+
|        Species present        |         Genes present         |
+-------------------------------+-------------------------------+
| [Acropora gemmifera, Acrop... |               []              |
|       [Acropora palmata]      |               []              |
|      [Acropora millepora]     |               []              |
|               []              |               []              |
|      [Acropora millepora]     | [DNA, DNA, DNA, DNA, DNA, ... |
|     [Acropora digitifera]     |               []              |
| [Acropora millepora, Acrop... |               []              |
|     [Acropora cervicornis]    |       [mRNA, mRNA, mRNA]      |
|               []              |    [Pol polyprotein, term]    |
|     [Acropora hyacinthus]     |               []              |
+-------------------------------+-------------------------------+
[73 rows x 9 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [18]:
def totalPapersWithGene(gene_record):
    total_papers = 0
    protein_name = gene_record['Protein names']
    gene_name = gene_record['Gene names']
    for gene_list in summary['Genes present']:
        if protein_name in gene_list or gene_name in gene_list:
            total_papers += 1
    return total_papers

In [19]:
gene_info = swissprot
gene_info['Total papers'] = gene_info.apply(totalPapersWithGene)
gene_info.materialize()
gene_info = gene_info.sort('Total papers', ascending=False)
totalPapersContainingGenes = sum(len(gene_list) > 0 for gene_list in summary['Genes present'])
gene_info[gene_info['Total papers'] > 0]['Gene names']

dtype: str
Rows: ?
['CMT2', 'Dnmt3a', 'DRM1A', 'Dnmt3c', 'CMT3', 'DMT1', 'dnmA', 'Dnmt3b', 'DNMT1', 'DNMT3A', 'MET4', 'MET3', 'MET2A', 'ZMET5', 'DRM2', 'MET1B', 'CMT1', 'Dnmt1', 'DNMT3B', 'MET2', 'DMT105', 'MET1A', 'DRM1', 'DNMT', 'term', 'impact', 'impact', 'pcif1', 'PCIF1', 'Pcif1', 'alpha', 'alpha', 'alpha', 'alpha', 'alpha', 'cope', 'NOP2C', 'HSP70', 'HSP70', 'HSP70', 'HSP70', 'HSP70', 'RPL37', 'QCR2', 'RPL3', 'RPS27A', 'RPS27A', 'RPS2', None, 'NLRC5', 'pdp1', 'RPS27A', 'GSE1', 'RPL3', 'adcyap1', 'Dgat1', 'CCT7', 'NAIP', 'rmtB', 'Hsf1', 'CCT7', 'RPS27A', 'star', None, 'bud23', 'ACTB', 'RPS2', None, 'hsf1', 'HSP90', 'hsp70', 'HSP90', 'kamB', 'beta', 'ACTB', 'fmrO', 'BUD23', 'RID2', 'star', 'TATA', 'RPL3', 'TUBB', 'NLRC3', 'ACTB', 'BTRC', 'EF1A', 'slim', 'RPS2', 'HSP90', 'rmtC', 'TUBB', 'csnk1d', 'npmA', None, ... ]

### Find loose matches hiding in a phrase like "wnt-like genes" or "ras-related proteins"

In [20]:
def findPossibleGeneTerms(text):
    return re.findall('\w+(?=-related|-like)', text)

In [21]:
summary['Possible gene terms'] = summary['Abstract'].apply(findPossibleGeneTerms)
summary[summary['Possible gene terms']]


Columns:
	ID	str
	DOI	str
	Title	str
	Date	str
	Keywords	str
	Abstract	str
	Language	str
	Species present	list
	Genes present	list
	Possible gene terms	list

Rows: Unknown

Data:
+-------------------------------+-------------------------------+
|               ID              |              DOI              |
+-------------------------------+-------------------------------+
|   10.1186_s12864-015-2355-x   |   10.1186/s12864-015-2355-x   |
|  10.1371_journal.pone.0016095 |  10.1371/journal.pone.0016095 |
|  10.1371_journal.pone.0081821 |  10.1371/journal.pone.0081821 |
|         10.1086_692717        |              None             |
|       10.3354_dao069079       |              None             |
| 10.1016_j.scitotenv.2019.0... | 10.1016/j.scitotenv.2019.0... |
|  10.1371_journal.pone.0013975 |  10.1371/journal.pone.0013975 |
|   10.1007_s12562-015-0862-y   |   10.1007/s12562-015-0862-y   |
|  10.1016_j.jembe.2012.06.020  |  10.1016/j.jembe.2012.06.020  |
|   10.1186_s12864-017-3959-0   |   10.1186/s12864-017-3959-0   |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------+
|             Title             |        Date       |
+-------------------------------+-------------------+
| Functional conservation of... |        None       |
| Regulation of Apoptotic Me... |  January 24, 2011 |
| RNA-seq Profiles of Immune... | November 21, 2013 |
| Early Transcriptional Resp... |    31 May 2017    |
| Bacterial community struct... |        None       |
| Science of the Total Envir... |    25 July 2019   |
| Gene Expression in the Scl... | November 12, 2010 |
| Possible involvement of Ta... |   18 March 2015   |
| Differential gene expressi... |    17 July 2012   |
| Transcriptomic analysis of... |        None       |
+-------------------------------+-------------------+
+-------------------------------+-------------------------------+----------+
|            Keywords           |            Abstract           | Language |
+-------------------------------+-------------------------------+----------+
| Apoptosis,Caspase,Bcl-2,Co... | Background: Apoptotic cell... |    en    |
|              None             | Background: Mass coral ble... |    en    |
|              None             | Coral diseases are among t... |    en    |
| HSP, heat shock protein,Lo... | Corals respond to heat pul... |    en    |
|              None             | Culture-independent molecu... |    en    |
| Benzo(a)pyrene Antioxidant... | The impact of acute benzo(... |    en    |
|              None             | Background: The success of... |    en    |
| N-acetyl-D-galactosamine,A... | N-acetyl-D-galactosamine, ... |    en    |
| Coral Gene expression HiCE... | We describe a strategy to ... |    en    |
| DMSP pathway,Acropora mill... | Background: Dimethylsulfon... |    en    |
+-------------------------------+-------------------------------+----------+
+--------------------------+-------------------------------+
|     Species present      |         Genes present         |
+--------------------------+-------------------------------+
|   [Acropora millepora]   |               []              |
|   [Acropora millepora]   | [DNA, DNA, DNA, DNA, DNA, ... |
|  [Acropora cervicornis]  |       [mRNA, mRNA, mRNA]      |
|            []            |    [Pol polyprotein, term]    |
|    [Acropora palmata]    |          [star, star]         |
|            []            |        [impact, impact]       |
| [Acropora microphthalma] | [DNA, DNA, DNA, DNA, DNA, ... |
|    [Acropora tenuis]     | [DNA, DNA, DNA, DNA, DNA, ... |
|    [Acropora tenuis]     | [DNA, DNA, DNA, DNA, DNA, ... |
|   [Acropora millepora]   |               []              |
+--------------------------+-------------------------------+
+------------------------------+
|     Possible gene terms      |
+------------------------------+
|         [vertebrate]         |
|            [3, 3]            |
|            [Toll]            |
|      [Krü

## TF-IDF K-Means Clustering and Nearest Neighbors
### (cool but admittedly not useful on the current dataset - leaving it just for fun)

In [44]:
def wordsOnly(text):
    text = re.sub(',', ' ', text)
    text = re.sub('[^a-zA-Z\s]', '', text)
    return text

In [45]:
summary = summary[summary['Language'] == 'en']
summary['Abstract_Words_Only'] = summary['Abstract'].apply(wordsOnly)

In [46]:
summary['tfidf'] = turicreate.text_analytics.tf_idf(summary['Abstract_Words_Only'])
summary['tfidf']

dtype: dict
Rows: 73
[{'communities': 2.344549292093078, 'understanding': 1.7255100836868542, 'more': 1.3460204619819507, 'analysis': 1.1549652252192415, 'species': 2.6920409239639014, 'primarily': 2.9041650800285006, 'impacting': 3.597312260588446, 'assumption': 4.290459441148391, 'rubisco': 4.290459441148391, 'anhydrase': 2.9041650800285006, 'remodelling': 4.290459441148391, 'interactions': 2.498699971920336, 'involved': 1.2459370034249682, 'regulation': 2.211017899468555, 'upregulation': 2.211017899468555, 'kaniewska': 4.290459441148391, 'members': 2.211017899468555, 'family': 2.211017899468555, 'bcl': 3.1918471524802814, 'eg': 3.597312260588446, 'antiapoptotic': 3.1918471524802814, 'suite': 4.290459441148391, 'hsps': 2.9041650800285006, 'heat': 2.800175366504453, 'upregulating': 3.597312260588446, 'acclimate': 4.290459441148391, 'capacity': 3.1918471524802814, 'rocker': 4.290459441148391, 'response': 0.8247235383486646, 'strong': 2.681021528714291, 'lacked': 3.597312260588446, 'alt

### Determine which article is the nearest neighbor of a given article (based on TF-IDF in abstract)

In [47]:
nn_model = turicreate.nearest_neighbors.create(summary, features=['tfidf'], label='ID')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [48]:
nn_model.query(summary[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 1.36986     | 697us        |

| Done         |         | 100         | 2.323ms      |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	str
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-------------------------------+--------------------+------+
| query_label |        reference_label        |      distance      | rank |
+-------------+-------------------------------+--------------------+------+
|      0      |   10.1038_s41598-018-21267-3  |        0.0         |  1   |
|      0      |      10.1242_bio.2011036      | 0.8264984227129337 |  2   |
|      0      | 10.1111_j.1365-294x.2011.0... | 0.8418079096045198 |  3   |
|      0      |  10.1016_j.margen.2015.08.001 | 0.8490566037735849 |  4   |
|      0      |  10.1371_journal.pone.0039099 | 0.8494623655913979 |  5   |
+-------------+-------------------------------+--------------------+------+
[5 rows x 4 columns]

### Plot SSE to determine reasonable number of clusters

In [ ]:
sses = []
for k in range(1, len(summary)):
    model = turicreate.kmeans.create(summary, features=['tfidf'], num_clusters=k)
    sses.append(sum(model.cluster_info['sum_squared_distance']))

Batch size is larger than the input dataset. Switching to an exact Kmeans method.

Choosing initial cluster centers with Kmeans++.

+---------------+-----------+

| Center number | Row index |

+---------------+-----------+

...

+-----------+-------------------------------+

| Iteration | Number of changed assignments |

+-----------+-------------------------------+

| 1         | 0                             |

+-----------+-------------------------------+

In [28]:
from matplotlib import pyplot as plt

In [43]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.plot(range(len(sses)),sses)

### Manually choose k, perform K-Means clustering

In [40]:
k = 21
kmeans_model = turicreate.kmeans.create(summary, features=['tfidf'], num_clusters=k)

Batch size is larger than the input dataset. Switching to an exact Kmeans method.

Choosing initial cluster centers with Kmeans++.

+---------------+-----------+

| Center number | Row index |

+---------------+-----------+

...

| Iteration | Number of changed assignments |

+-----------+-------------------------------+

| 1         | 1                             |

| 2         | 0                             |

+-----------+-------------------------------+

In [41]:
kmeans_model.cluster_info.print_rows(num_rows=k)

+-------------------------------+------------+------+----------------------+
|             tfidf             | cluster_id | size | sum_squared_distance |
+-------------------------------+------------+------+----------------------+
| {'a': 0.2573004652722753, ... |     0      |  1   |         0.0          |
| {'a': 0.4288341087871255, ... |     1      |  1   |         0.0          |
| {'abundant': 3.59731226058... |     2      |  1   |         0.0          |
| {'a': 0.3430672870297004, ... |     3      |  1   |         0.0          |
| {'a': 0.3430672870297004, ... |     4      |  1   |         0.0          |
| {'a': 0.6861345740594008, ... |     5      |  1   |         0.0          |
| {'a': 0.3130488994146017, ... |     6      |  40  |  72380.57173156738   |
| {'a': 0.3259139226782154, ... |     7      |  10  |  15396.830017089844  |
| {'a': 0.3430672870297004, ... |     8      |  1   |         0.0          |
| {'a': 0.857668217574251, '... |     9      |  1   |         0.0          |

### What are the top 10 words defining a cluster?

In [42]:
info = kmeans_model.cluster_info
for k in range(0, len(info)):
    cluster = info[info['cluster_id'] == k]
    top_words_tfidf = cluster[['tfidf']].stack('tfidf', new_column_name=['word','tfidf']).sort('tfidf', ascending=False)
    top_words = top_words_tfidf['word'][0:10]
    print(f"Cluster {k}")
    print(top_words)

Cluster 0
['winter', 'collected', 'summer', 'otsuki', 'cold', 'muricata', 'kochi', 'photochemical', 'health', 'stress']
Cluster 1
['aws', 'isolates', 'characterized', 'vibrio', 'clone', 'revealing', 'diseased', 'healthy', 'rrna', 'libraries']
Cluster 2
['cloud', 'duplicated', 'warm', 'environments', 'diverse', 'periods', 'belonging', 'survived', 'marine', 'ancestor']
Cluster 3
['selection', 'homologous', 'homologues', 'dnds', 'pairs', 'ratios', 'datasets', 'contigs', 'similarities', 'independent']
Cluster 4
['go', 'terms', 'nlrclike', 'groups', 'protein', 'overrepresented', 'pairedend', 'ubiquitination', 'aculeus', 'nfkb']
Cluster 5
['postfertilization', 'c', 'development', 'palmata', 'division', 'replicationcell', 'embryos', 'stages', 'embryonic', 'temperatures']
Cluster 6
['disease', 'domain', 'thermal', 'symbiodinium', 'stress', 'dmsp', 'colonies', 'response', 'immune', 'elevated']
Cluster 7
['et', 'al', 'stressresponsive', 'fastigiata', 'bleaching', 'reference', 'protein', 'irradia